# 5. Execution and Model Management


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">
このコードの部分について、ソクラテス式チャットボットの開発を例に説明させていただきます。

この部分は大きく分けて3つの重要な処理を行っています：

### 1. データの分割 (5.1 Data Set Split and Validation)
```python
dataset_size = len(tokenized_dataset)
indices = np.random.permutation(dataset_size)
split_idx = int(dataset_size * 0.8)
```
これは、用意した会話データを「学習用」と「評価用」に分ける作業です。

例えば、1000個の会話データがあった場合：
- 800個（80%）を学習用データとして使用
- 残りの中から最大100個を評価用データとして使用

評価用データは、モデルが本当にソクラテス風の話し方を身につけているか確認するために使います。

### 2. トレーニングの準備と実行 (5.2 Training Execution Control)
```python
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    ...
)
```
これは、実際の学習を行うための準備です。例えると、以下のような設定を行っています：
- モデル（生徒）に「かね？」という語尾の使い方を教える
- 質問で返すパターンを学ばせる
- 「では、そこで君に問おう」といった老練な言い回しを覚えさせる

### 3. チェックポイントと保存の管理
```python
if os.path.exists(checkpoint_dir):
    logging.info("\nChecking checkpoint status...")
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")]
```
これは、学習の途中経過を保存する仕組みです。

例えば：
- 10時間の学習予定で、2時間ごとに進捗を保存
- 途中で停電などが起きても、最後に保存した地点から再開可能
- 学習が完了したら、ソクラテス風の話し方を習得したモデルを保存

実際の例で言うと：
```
モデル「なるほど、君は研究での成功を喜びと感じるようだね」
↓ （学習中に停電）
↓ （前回の保存地点から再開）
↓ （学習完了）
モデル「では、その喜びと幸せは、何か違いがあるのかね？」
```

このように、長時間の学習プロセスを安全に管理しながら、目指すソクラテス風の話し方を習得させていく仕組みになっています。


        
</div>
    </details>
</div>


# 5.1 Data Set Split and Validation

In [ ]:
# 5.1 Data Set Split and Validation
# Data set split
dataset_size = len(tokenized_dataset)
indices = np.random.permutation(dataset_size)
split_idx = int(dataset_size * 0.8)

# Create training and test datasets
train_dataset = tokenized_dataset.select(indices[:split_idx])
eval_dataset = tokenized_dataset.select(indices[split_idx:split_idx+100])  # Maximum 100 samples

# Record dataset size
logging.info(f"Training dataset size: {len(train_dataset)}")
logging.info(f"Evaluation dataset size: {len(eval_dataset)}")


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

はい、この部分について、ソクラテス式チャットボットの開発に即して、より詳しく説明させていただきます。

### コードの詳細説明

```python
# データセットの総数を取得
dataset_size = len(tokenized_dataset)

# データをランダムに並び替えるための準備
indices = np.random.permutation(dataset_size)

# 80%のデータを学習用とするための分割点を計算
split_idx = int(dataset_size * 0.8)
```

これは、会話データを「学習用」と「評価用」に分ける処理です。具体的な例を使って説明します：

例えば、以下のような10,000個の会話データがあるとします：

```json
[
  {
    "messages": [
      {
        "role": "user",
        "content": "幸せとは何だと思いますか？"
      },
      {
        "role": "model",
        "content": "君にとって幸せとは何かを、まず考えてみてはどうかね？"
      }
    ]
  },
  // ... 他の会話データ ...
]
```

このデータを以下のように分割します：

```python
# 学習用と評価用のデータセットを作成
train_dataset = tokenized_dataset.select(indices[:split_idx])
eval_dataset = tokenized_dataset.select(indices[split_idx:split_idx+100])
```

1. **学習用データ（train_dataset）**
   - 全体の80%（この例では8,000個）の会話を使用
   - モデルが「ソクラテス風の話し方」を学ぶための教材
   - 例：
     - 「〜かね？」という語尾の使い方
     - 質問で返す話し方
     - 「では、君はどう考えるのかね？」といった問いかけ方

2. **評価用データ（eval_dataset）**
   - 残りのデータから最大100個を使用
   - モデルが正しく学習できているか確認するためのテスト問題
   - 例：
     - ユーザー：「研究が思うように進まなくて悩んでいます」
     - 期待する応答：「なるほど。では君は、その研究の何が思うように進まないと感じているのかね？」

```python
# データセットのサイズを記録
logging.info(f"Training dataset size: {len(train_dataset)}")
logging.info(f"Evaluation dataset size: {len(eval_dataset)}")
```

この部分は、分割したデータの数を記録します。これにより：
- 実際に何個のデータで学習するのか
- 何個のデータでテストするのか
が明確になり、後で学習結果を評価する際の参考になります。

このような分割が必要な理由は、「暗記」と「理解」を区別するためです。学習用データだけで評価すると、モデルが単に会話を「暗記」しただけなのか、本当に「ソクラテス風の話し方」を「理解」したのか区別できません。評価用の新しいデータで試すことで、モデルが本当にソクラテス風の対話の特徴を学習できているか確認できるのです。

        
</div>
    </details>
</div>


# 5.2 Training Execution Control

In [ ]:
# 5.2 Training Execution Control
# Trainer initialization
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[StyleCallback(), TrainingMonitorCallback()],
)

# Check memory state
log_memory_usage()

# Training execution
logging.info("Starting training...")
try:
    checkpoint_dir = "./model"
    resume_from_checkpoint = None
    
    # Checkpoint status and processing modification
    if os.path.exists(checkpoint_dir):
        logging.info("\nChecking checkpoint status...")
        checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")]
        if checkpoints:
            # Get latest checkpoint
            latest_checkpoint = max(checkpoints, key=lambda x: int(x.split("-")[1]))
            checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
            logging.info(f"Found latest checkpoint: {latest_checkpoint}")
            
            # Check checkpoint status
            state_path = os.path.join(checkpoint_path, "trainer_state.json")
            if os.path.exists(state_path):
                with open(state_path, 'r') as f:
                    state = json.load(f)
                current_epoch = state.get('epoch', 0)
                logging.info(f"\nCurrent training status:")
                logging.info(f"Current epoch: {current_epoch}")
                logging.info(f"Target epochs: {training_args.num_train_epochs}")
                
                # Exit safely if completed
                if current_epoch >= training_args.num_train_epochs - 0.1:
                    logging.info("\n" + "="*50)
                    logging.info("IMPORTANT NOTICE:")
                    logging.info(f"Training has already been completed at epoch {current_epoch}!")
                    logging.info(f"Target epochs was {training_args.num_train_epochs}")
                    logging.info(f"Trained model is available at: {checkpoint_dir}")
                    logging.info("="*50 + "\n")
                    exit(0)
            else:
                logging.warning("Invalid checkpoint state found. Proceeding with training...")
                logging.warning(f"Checkpoint directory: {checkpoint_dir}")
        else:
            logging.warning("Checkpoint directory exists but no checkpoints found. Proceeding with training...")

    # Start learning (or resume)
    trainer.train(resume_from_checkpoint=resume_from_checkpoint)
    logging.info("Training completed successfully!")
    
    # Save settings (as JSON)
    import json

    def convert_to_serializable(obj):
        if isinstance(obj, set):
            return list(obj)
        elif isinstance(obj, dict):
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [convert_to_serializable(x) for x in obj]
        return obj

    # Convert all settings
    training_args_dict = convert_to_serializable(training_args.to_dict())
    lora_config_dict = convert_to_serializable(lora_config.to_dict())

    config_dict = {
        "model_name": model_name,
        "training_args": training_args_dict,
        "lora_config": lora_config_dict,
        "bnb_config": {
            "load_in_4bit": bnb_config.load_in_4bit,
            "bnb_4bit_use_double_quant": bnb_config.bnb_4bit_use_double_quant,
            "bnb_4bit_quant_type": bnb_config.bnb_4bit_quant_type,
            "bnb_4bit_compute_dtype": str(bnb_config.bnb_4bit_compute_dtype),
        }
    }
    
    with open(os.path.join(training_args.output_dir, "training_config.json"), "w", encoding="utf-8") as f:
        json.dump(config_dict, f, indent=2, ensure_ascii=False)
    
    # Save model
    trainer.save_model()
    # Save settings
    model.config.save_pretrained(training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)
    logging.info("Model and configuration saved successfully!")

except Exception as e:
    logging.error(f"An error occurred: {str(e)}")
    # Checkpoint is also kept even on error
    raise 




<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



この部分について、ソクラテス式チャットボットの開発プロジェクトに即して説明させていただきます。大きく分けて4つの重要な部分があります：

### 1. トレーナーの初期化
```python
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[StyleCallback(), TrainingMonitorCallback()],
)
```
これは学習の「先生役」を設定する部分です。具体的には：
- `StyleCallback`: ソクラテス風の話し方（「〜かね？」「では、君は〜」など）が properly できているか監視
- `TrainingMonitorCallback`: 学習の進み具合を監視し、グラフなどで可視化

### 2. チェックポイントの管理
```python
checkpoint_dir = "./model"
if os.path.exists(checkpoint_dir):
    logging.info("\nChecking checkpoint status...")
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")]
```
これは学習の途中経過を保存・管理する部分です。例えば：

- 1000回の対話練習のうち、100回ごとに進捗を保存
- もし200回目で停電が起きても、100回目の状態から再開可能
- 既に完了している場合（例：1000回終わっている）は、再度学習を開始しない

### 3. 学習の実行
```python
trainer.train(resume_from_checkpoint=resume_from_checkpoint)
logging.info("Training completed successfully!")
```
実際の学習を行う部分です。例えば：
```
ユーザー: 「幸せとは何でしょうか？」
モデル: 「そうだね...」（← まだソクラテス風ではない）
↓ （学習を重ねる）
モデル: 「君にとって幸せとは何かを、まず考えてみてはどうかね？」（← ソクラテス風に進化）
```

### 4. 設定と成果物の保存
```python
# Save settings (as JSON)
config_dict = {
    "model_name": model_name,
    "training_args": training_args_dict,
    "lora_config": lora_config_dict,
    ...
}
```
学習が終わった後の処理です：
- 学習したモデルを保存（後で使えるように）
- 学習設定を保存（どういう設定で学習したか記録）
- トークナイザーを保存（言葉の解釈方法を保存）

例えると、以下のようなものを保存：
- ソクラテス風の話し方のパターン
- 「〜かね？」という語尾の使い方
- 質問で返すタイミング
- 「では、そこで君に問おう」といった特徴的なフレーズの使い方

このように、学習全体のプロセスを管理し、途中で問題が起きても安全に再開でき、最終的な成果物をきちんと保存する仕組みになっています。

        
</div>
    </details>
</div>
